# Informe a data ou deixe em branco para pegar as últimas atividades

Exemplos:

date_act = "26/07/2021"

date_act = ""

In [2]:
#Informa a data da atividade
#date_act = "2021-06-19"
date_act = ""

# Prep Environment

In [3]:
!pip install garminconnect
!pip install mysql-connector-python

In [4]:
import numpy as np
import pandas as pd
import mysql.connector

#Conecta com banco MySQL
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='lab-triathlon',
                                         user='root',
                                         password='12345678')
    
    sql_select_Query = " select garmin_login, garmin_pass from athletes "
    
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    
    #carrega dados
    records = cursor.fetchall()

    data_sql = pd.DataFrame(records)

except mysql.connector.Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if connection.is_connected():
        connection.close()
        cursor.close()
        print("MySQL connection is closed")
        
data_sql.info()

MySQL connection is closed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2 non-null      object
 1   1       2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [5]:
df_atl = data_sql.copy()

df_atl.rename(columns={0: "login", 1: "pass"}, inplace=True)

df_atl

,login,pass
0,rafaelcostaleite@gmail.com,Rmcl_g19@
1,betolundgren@yahoo.com.br,Atleta10


In [6]:
#!/usr/bin/env python3

from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)

from datetime import date
from datetime import datetime

"""
Enable debug logging
"""
import logging
logging.basicConfig(level=logging.DEBUG)
#logging.basicConfig(level=logging.CRITICAL)

today = date.today()

# App

In [9]:
for ind in df_atl.index:
  #dados_atleta = garmin_dados_atividade(df_atl['e-mail'][ind],df_atl['senha'][ind])
    YOUR_EMAIL = df_atl['login'][ind]
    YOUR_PASSWORD = df_atl['pass'][ind]

    """
    Initialize Garmin client with credentials
    Only needed when your program is initialized
    """
    print("Garmin(email, password)")
    print("----------------------------------------------------------------------------------------")
    print("Atleta:" + YOUR_EMAIL)
    try:
      client = Garmin(YOUR_EMAIL, YOUR_PASSWORD)
    except (
      GarminConnectConnectionError,
      GarminConnectAuthenticationError,
      GarminConnectTooManyRequestsError,
    ) as err:
        print("Não foi possível iniciar o client: %s" % err)
        quit()
    except Exception:  # pylint: disable=broad-except
        print("Unknown error occurred during Garmin Connect Client init")
        quit()

    """
    Login to Garmin Connect portal
    Only needed at start of your program
    The library will try to relogin when session expires
    """
    print("client.login()")
    print("----------------------------------------------------------------------------------------")
    try:
        client.login()
    except (
      GarminConnectConnectionError,
      GarminConnectAuthenticationError,
      GarminConnectTooManyRequestsError,
    ) as err:
        print("Não foi possível efetuar login: %s" % err)
        quit()
    except Exception:  # pylint: disable=broad-except
        print("Unknown error occurred during Garmin Connect Client login")
        quit()

    """
    Get unit system from profile
    """
    units = "Não informado"

    print("client.get_unit_system()")
    print("----------------------------------------------------------------------------------------")
    try:
        units = client.get_unit_system()
    except (
      GarminConnectConnectionError,
      GarminConnectAuthenticationError,
      GarminConnectTooManyRequestsError,
    ) as err:
        print("Error occurred during Garmin Connect Client get unit system: %s" % err)
        quit()
    except Exception:  # pylint: disable=broad-except
        print("Unknown error occurred during Garmin Connect Client get unit system")
        quit()

    """
    Get activities data
    """
    print("client.get_activities()")
    print("----------------------------------------------------------------------------------------")
    try:
        activities = client.get_activities(0,100) # 0=start, 1=limit
    except (
      GarminConnectConnectionError,
      GarminConnectAuthenticationError,
      GarminConnectTooManyRequestsError,
    ) as err:
        print("Error occurred during Garmin Connect Client get activities: %s" % err)
        quit()
    except Exception:  # pylint: disable=broad-except
        print("Unknown error occurred during Garmin Connect Client get activities")
        quit()

    if activities is not None:
        df_act = pd.DataFrame(activities)
    else:
        df_act = pd.DataFrame()

    """
    Get stats and body composition data
    """
    print("client.get_stats_and_body_composition(%s)", today.isoformat())
    print("----------------------------------------------------------------------------------------")
    try:
        bodycom = client.get_stats_and_body(today.isoformat())
      #bodycom = (client.get_body_composition(today.isoformat()))
    except (
      GarminConnectConnectionError,
      GarminConnectAuthenticationError,
      GarminConnectTooManyRequestsError,
    ) as err:
        print("Error occurred during Garmin Connect Client get stats and body composition: %s" % err)
        quit()
    except Exception:  # pylint: disable=broad-except
        print("Unknown error occurred during Garmin Connect Client get stats and body composition")
        quit()

    df_result = pd.DataFrame()

    if len(df_act) > 0:
        df_type = pd.json_normalize(df_act['activityType'])

        df_result = pd.concat([df_act, df_type], axis=1)
        df_result.drop(columns=['activityType', 'eventType'],inplace=True)

    if type(bodycom['weight']) == float:
        df_result['weight'] = ((bodycom['weight'])/1000)
    else:
        df_result['weight'] = 0

    df_result['units'] = units

    del df_act
    del activities
    del bodycom
    del units

    if ind == 0:
        dados_atividades = df_result
    else:
        dados_atividades = pd.concat([dados_atividades,df_result])

DEBUG:garminconnect:Login to Garmin Connect using POST url https://sso.garmin.com/sso/signin
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sso.garmin.com:443


Garmin(email, password)
----------------------------------------------------------------------------------------
Atleta:rafaelcostaleite@gmail.com
client.login()
----------------------------------------------------------------------------------------


DEBUG:urllib3.connectionpool:https://sso.garmin.com:443 "GET /sso/signin?webhost=https%3A%2F%2Fconnect.garmin.com&service=https%3A%2F%2Fconnect.garmin.com&source=https%3A%2F%2Fsso.garmin.com%2Fsso%2Fsignin&redirectAfterAccountLoginUrl=https%3A%2F%2Fconnect.garmin.com&redirectAfterAccountCreationUrl=https%3A%2F%2Fconnect.garmin.com&gauthHost=https%3A%2F%2Fsso.garmin.com%2Fsso&locale=en_US&id=gauth-widget&cssUrl=https%3A%2F%2Fstatic.garmincdn.com%2Fcom.garmin.connect%2Fui%2Fcss%2Fgauth-custom-v1.2-min.css&clientId=GarminConnect&rememberMeShown=true&rememberMeChecked=false&createAccountShown=true&openCreateAccount=false&usernameShown=false&displayNameShown=false&consumeServiceTicket=false&initialFocus=true&embedWidget=false&generateExtraServiceTicket=false HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://sso.garmin.com:443 "POST /sso/signin?webhost=https%3A%2F%2Fconnect.garmin.com&service=https%3A%2F%2Fconnect.garmin.com&source=https%3A%2F%2Fsso.garmin.com%2Fsso%2Fsignin&redirectAf

client.get_unit_system()
----------------------------------------------------------------------------------------
client.get_activities()
----------------------------------------------------------------------------------------


DEBUG:urllib3.connectionpool:https://connect.garmin.com:443 "GET /proxy/activitylist-service/activities/search/activities?start=0&limit=100 HTTP/1.1" 200 None
DEBUG:garminconnect:Fetch response code 200
DEBUG:garminconnect:Fetch response json [{'activityId': 7170525094, 'activityName': 'Zwift - Team Italy Specialissima Race (C)', 'description': None, 'startTimeLocal': '2021-07-22 11:40:13', 'startTimeGMT': '2021-07-22 14:40:13', 'activityType': {'typeId': 152, 'typeKey': 'virtual_ride', 'parentTypeId': 2, 'sortOrder': 80, 'isHidden': False, 'restricted': False}, 'eventType': {'typeId': 9, 'typeKey': 'uncategorized', 'sortOrder': 10}, 'comments': None, 'parentId': None, 'distance': 40822.8515625, 'duration': 3934.0, 'elapsedDuration': 3932.0, 'movingDuration': 3928.0, 'elevationGain': 438.0, 'elevationLoss': 0.0, 'averageSpeed': 10.37600040435791, 'maxSpeed': 17.05900001525879, 'startLatitude': 0.0, 'startLongitude': 0.0, 'hasPolyline': True, 'ownerId': 16372755, 'ownerDisplayName': 'ra

...

Não foi possível efetuar login: Error connecting
client.get_unit_system()
----------------------------------------------------------------------------------------
client.get_activities()
----------------------------------------------------------------------------------------


DEBUG:urllib3.connectionpool:https://connect.garmin.com:443 "GET /proxy/activitylist-service/activities/search/activities?start=0&limit=100 HTTP/1.1" 403 None
DEBUG:garminconnect:Fetch response code 403
DEBUG:garminconnect:Exception occurred during data retrieval - perhaps session expired - trying relogin: 403 Client Error: Forbidden for url: https://connect.garmin.com/proxy/activitylist-service/activities/search/activities?start=0&limit=100
DEBUG:garminconnect:Login to Garmin Connect using POST url https://sso.garmin.com/sso/signin
DEBUG:urllib3.connectionpool:https://sso.garmin.com:443 "GET /sso/signin?webhost=https%3A%2F%2Fconnect.garmin.com&service=https%3A%2F%2Fconnect.garmin.com&source=https%3A%2F%2Fsso.garmin.com%2Fsso%2Fsignin&redirectAfterAccountLoginUrl=https%3A%2F%2Fconnect.garmin.com&redirectAfterAccountCreationUrl=https%3A%2F%2Fconnect.garmin.com&gauthHost=https%3A%2F%2Fsso.garmin.com%2Fsso&locale=en_US&id=gauth-widget&cssUrl=https%3A%2F%2Fstatic.garmincdn.com%2Fcom.garmin

Error occurred during Garmin Connect Client get activities: Error connecting
client.get_stats_and_body_composition(%s) 2021-07-23
----------------------------------------------------------------------------------------


DEBUG:urllib3.connectionpool:https://connect.garmin.com:443 "GET /proxy/usersummary-service/usersummary/daily/?calendarDate=2021-07-23 HTTP/1.1" 405 71
DEBUG:garminconnect:Statistics response code 405


Error occurred during Garmin Connect Client get stats and body composition: Error connecting


In [8]:
dados_atividades

,activityId,activityName,description,startTimeLocal,startTimeGMT,comments,parentId,distance,duration,elapsedDuration,...,typeId,typeKey,parentTypeId,sortOrder,isHidden,restricted,weight,units,moderateIntensityMinutes,vigorousIntensityMinutes
0,7170525094,Zwift - Team Italy Specialissima Race (C),None,2021-07-22 11:40:13,2021-07-22 14:40:13,None,None,40822.851562,3934.000000,3932.000000,...,152,virtual_ride,2,80,False,False,0,metric,NaN,NaN
1,7170170014,Zwift - NORSEMAN Festival of Climbing Workout ...,None,2021-07-22 11:00:47,2021-07-22 14:00:47,None,None,15044.019531,2113.363037,2111.000000,...,152,virtual_ride,2,80,False,False,0,metric,NaN,NaN
2,7158950312,Rio de Janeiro Corrida,None,2021-07-20 13:23:31,2021-07-20 16:23:31,None,None,747.429993,346.292999,346.292999,...,1,running,17,3,False,False,0,metric,NaN,NaN
3,7158949727,Rio de Janeiro Corrida,None,2021-07-20 12:25:42,2021-07-20 15:25:42,None,None,10022.540039,3449.373047,3449.373047,...,1,running,17,3,False,False,0,metric,NaN,NaN
4,7135052729,Rio de Janeiro Corrida,None,2021-07-16 08:34:18,2021-07-16 11:34:18,None,None,11233.269531,3864.481934,3871.548096,...,1,running,17,3,False,False,0,metric,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6783436849,Rio de Janeiro Ciclismo,None,2021-05-15 05:58:42,2021-05-15 08:58:42,None,None,105989.539062,13722.868164,15036.156250,...,2,cycling,17,8,False,False,0,metric,NaN,NaN
96,6776911874,Rio de Janeiro Corrida,None,2021-05-14 06:53:29,2021-05-14 09:53:29,None,None,9223.339844,2838.839111,3030.517090,...,1,running,17,3,False,False,0,metric,NaN,NaN
97,6771939694,Natação em piscina,None,2021-05-13 09:05:36,2021-05-13 12:05:36,None,None,2000.000000,2553.900879,2580.175049,...,27,lap_swimming,26,25,False,False,0,metric,NaN,NaN
98,6771618549,Ciclismo em local fechado,None,2021-05-13 07:22:33,2021-05-13 10:22:33,None,None,0.000000,5431.757812,5431.757812,...,25,indoor_cycling,2,11,False,False,0,metric,NaN,NaN


In [19]:
#Copia
selected_columns  = dados_atividades[["ownerDisplayName","ownerFullName","startTimeGMT","startTimeLocal","units","distance","duration","elapsedDuration","movingDuration","elevationGain","elevationLoss","averageSpeed","maxSpeed","startLatitude","startLongitude","calories","averageHR","maxHR","averageBikingCadenceInRevPerMinute","maxBikingCadenceInRevPerMinute","minActivityLapDuration","typeKey"]]

dados_eventos = selected_columns.copy()

dados_eventos.fillna(0)

#Filtra data
start_date = date_act + " 00:00:00"
end_date = date_act + " 23:59:00"

if date_act:
  after_start_date = dados_eventos["startTimeLocal"] >= start_date
  before_end_date = dados_eventos["startTimeLocal"] <= end_date
  between_two_dates = after_start_date & before_end_date
  dados_eventos = dados_eventos.loc[between_two_dates]

#Campos tratados
dados_eventos['Tipo de atividade'] = dados_eventos['typeKey']
dados_eventos['Distancia Km'] = round(dados_eventos['distance']/1000,2)
dados_eventos['Distancia Metros'] = round(dados_eventos['distance'],2)

def convertehora(seconds):
	min, sec = divmod(seconds, 60)
	hour, min = divmod(min, 60)
	return "%d:%02d:%02d" % (hour, min, sec)
 
def convertespeed(velocidade):
  newvel = round((velocidade) * (60*60)/1000,2)
  return newvel

def convertpace(distancia,velocidade):
  pace = distancia/velocidade
  return pace

def convertpace(distancia,tempo):
  if tempo > 0  and distancia > 0:
    #pace = convertehora(tempo/distancia)
    pace = (tempo/distancia*60)
  else:
    pace = 0
  return pace

dados_eventos['duration'] = dados_eventos['duration'].fillna(0)
dados_eventos['elapsedDuration'] = dados_eventos['elapsedDuration'].fillna(0)
dados_eventos['movingDuration'] = dados_eventos['movingDuration'].fillna(0)

#dados_eventos['Tempo em Horas'] = dados_eventos['duration'].apply(convertehora)
#dados_eventos['Tempo Decorrido em Horas'] = dados_eventos['elapsedDuration'].apply(convertehora)
#dados_eventos['Tempo Movimento em Horas'] = dados_eventos['movingDuration'].apply(convertehora)
dados_eventos['Tempo Atividade'] = dados_eventos['duration']
dados_eventos['Tempo em Movimento'] = dados_eventos['movingDuration']

dados_eventos['Velocidade Média Km/h'] = dados_eventos['averageSpeed'].apply(convertespeed)
dados_eventos['Velocidade Máxima Km/h'] = dados_eventos['maxSpeed'].apply(convertespeed)

dados_eventos['Pace duração Km/min'] = dados_eventos.apply(lambda x: convertpace(x['Distancia Km'], x['duration']), axis=1)
dados_eventos['Pace movimento Km/min'] = dados_eventos.apply(lambda x: convertpace(x['Distancia Km'], x['movingDuration']), axis=1)

dados_eventos['Ganho de Elevação'] = dados_eventos['elevationGain']
dados_eventos['Perda de Elevação'] = dados_eventos['elevationLoss']

dados_eventos['Batimentos Média'] = dados_eventos['averageHR']
dados_eventos['Batimentos Máximo'] = dados_eventos['maxHR']

dados_eventos['Calorias'] = dados_eventos['calories']

dados_eventos['Bike Cadência Média por Minuto'] = dados_eventos['averageBikingCadenceInRevPerMinute']
dados_eventos['Bike Cadência Máxima por Minuto'] = dados_eventos['maxBikingCadenceInRevPerMinute']

dados_eventos.drop(['calories','minActivityLapDuration','maxBikingCadenceInRevPerMinute','distance', 'duration','elapsedDuration','movingDuration','averageSpeed','maxSpeed','typeKey','elevationGain','elevationLoss','averageHR','maxHR','averageBikingCadenceInRevPerMinute'], axis=1, inplace=True)

dados_eventos

,ownerDisplayName,ownerFullName,startTimeGMT,startTimeLocal,units,startLatitude,startLongitude,Tipo de atividade,Distancia Km,Distancia Metros,Tempo Atividade,Tempo em Movimento,Velocidade Média Km/h,Velocidade Máxima Km/h,Pace duração Km/min,Pace movimento Km/min,Ganho de Elevação,Perda de Elevação,Batimentos Média,Batimentos Máximo,Calorias,Bike Cadência Média por Minuto,Bike Cadência Máxima por Minuto
0,rafaelcostaleite,Rafael,2021-07-14 13:26:11,2021-07-14 10:26:11,metric,0.000000,0.000000,virtual_ride,32.44,32436.42,3319.693115,3300.000000,35.17,54.34,6139.999597,6103.575832,219.0,0.0,125.0,166.0,530.0,65.0,80.0
1,rafaelcostaleite,Rafael,2021-07-12 20:56:35,2021-07-12 17:56:35,metric,0.000000,0.000000,virtual_ride,30.60,30597.76,3324.425049,3302.000000,33.13,39.09,6518.480488,6474.509804,46.0,0.0,117.0,129.0,422.0,61.0,66.0
2,rafaelcostaleite,Rafael,2021-07-11 14:09:50,2021-07-11 11:09:50,metric,-22.952436,-43.196467,cycling,24.72,24715.62,5097.818848,4806.000000,17.45,46.66,12373.346718,11665.048544,614.0,601.0,150.0,195.0,603.0,NaN,NaN
3,rafaelcostaleite,Rafael,2021-07-10 13:47:27,2021-07-10 10:47:27,metric,-22.948371,-43.180767,running,10.62,10619.80,3497.847900,3492.982986,10.93,14.38,19761.852545,19734.367155,18.0,16.0,155.0,198.0,496.0,NaN,NaN
4,rafaelcostaleite,Rafael,2021-06-27 11:29:46,2021-06-27 08:29:46,metric,-22.948388,-43.180728,running,6.81,6808.21,2413.965088,2349.984011,10.15,14.54,21268.414871,20704.704940,9.0,9.0,154.0,188.0,335.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,betolundgren,betolundgren,2020-04-22 09:02:11,2020-04-22 06:02:11,metric,0.000000,0.000000,virtual_ride,35.93,35930.56,6131.794922,5971.000000,21.09,86.25,10239.568475,9971.054829,638.0,0.0,0.0,0.0,794.0,66.0,83.0
496,betolundgren,betolundgren,2020-04-20 09:06:38,2020-04-20 06:06:38,metric,0.000000,0.000000,virtual_ride,43.76,43761.47,6470.704102,6419.000000,24.35,77.18,8872.080578,8801.188300,599.0,0.0,0.0,0.0,1050.0,81.0,96.0
497,betolundgren,betolundgren,2020-04-20 00:23:57,2020-04-19 21:23:57,metric,NaN,NaN,indoor_cycling,0.00,0.00,6003.038086,0.000000,0.00,NaN,0.000000,0.000000,0.0,0.0,119.0,135.0,828.0,67.0,83.0
498,betolundgren,betolundgren,2020-04-18 00:28:48,2020-04-17 21:28:48,metric,NaN,NaN,indoor_cycling,0.00,0.00,6540.348145,0.000000,0.00,NaN,0.000000,0.000000,0.0,0.0,133.0,152.0,1093.0,82.0,96.0


In [20]:
#Exporta para excel
#now = datetime.now()
#now = now.strftime("%Y-%m-%d_%H%M%S")

#namefile = "blc-dados-atividades-" +now+ ".xlsx"

#dados_eventos.to_excel(namefile) 

#from google.colab import files
#files.download(namefile)

In [22]:
# authenticate
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

# create, and save df
from gspread_dataframe import set_with_dataframe

title = 'BLC-Dados'

gc.create(title)  # if not exist
sheet = gc.open(title).sheet1

set_with_dataframe(sheet, dados_eventos) 
# include_index=False, include_column_header=True, resize=False